# Boosting Algerian Agricultural Company Profits

<font color="#fcd695"> The goal from this project in to create a model that can predict the improvement of salles of agricultural products for a specific agricultural company in other words, we want to dectect how to boost the profit of Algerian Agricultural Company </strong></font>


## 1 Steps Of The Project:

### 1.1 Understand the Business/Research Context:

<font color="#ffe9c4">Gain a deep understanding of the business or research context. Understand the domain, business goals, and specific challenges.</font>

### 1.2 Data presentation:

<font color="#ffe9c4">
The dataset is csv file that contains a comprehensive dataset capturing various transactions within an agriculture company.The dataset has this features: 


 </font>

 | Column                  | Description                                              |
|-------------------------|----------------------------------------------------------|
| CleEffet                | Identifier for transactions.                             |
| Date                    | Date of the transaction.                                  |
| TypeEffet               | Type of transaction.                                      |
| FamilleArticle          | Family/category of the transaction product.              |
| CodeProduit             | Product code.                                            |
| ReferenceProduit        | Reference code for the product.                           |
| NLot                    | Lot number.                                              |
| Produit_Designation     | Original designation of the product of a transaction detail.|
| DetailEffet_Designation | Designation (sometimes different from Produit_Designation) of a transaction detail.|
| Quantite                | Quantity of the product.                                  |
| PrixAchat               | Purchase price.                                         |
| PrixUnitaireTTC         | Unit price including taxes.                              |
| DetailEffet_Marge       | Margin for the transaction detail.                        |
| Effet_Marge             | Margin for the transaction.                               |
| MontantHT               | Transaction amount excluding taxes.                      |
| Payement                | Payment amount.                                          |



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("./Project_datamining_final_data_03_12_an1.csv")
print(data.info())
print(data.shape)

In [ ]:
data.head()

In [ ]:
# Data Types Assingment
data['CleEffet'] = data['CleEffet'].astype(str)
data['Date'] = pd.to_datetime(data['Date'], format = '%m/%d/%Y')
data['TypeEffet'] = data['TypeEffet'].astype(str)
data['FamilleArticle'] = data['FamilleArticle'].astype(str)
data['Code Produit'] = data['Code Produit'].astype(str)
data['Reference Produit'] = data['Reference Produit'].astype(str)
data['NLot'] = data['NLot'].astype(str)
data['Produit_Designation'] = data['Produit_Designation'].astype(str)
data['DetailEffet_Designation'] = data['DetailEffet_Designation'].astype(str)
data['Code Produit'] = data['Code Produit'].astype(str)

data['PrixAchat'] = data['PrixAchat'].str.replace(',', '').astype('float64')
data['PrixUnitaireTTC'] = data['PrixUnitaireTTC'].str.replace(',', '').astype('float64')
data['DetailEffet_Marge'] = data['DetailEffet_Marge'].str.replace(',', '').astype('float64')
data['Effet_Marge'] = data['Effet_Marge'].str.replace(',', '').astype('float64')
data['MontantHT'] = data['MontantHT'].str.replace(',', '').astype('float64')
data['Payement'] = data['Payement'].str.replace(',', '').astype('float64')

In [ ]:
data['TypeEffet'].value_counts()

In [ ]:
data2013 = data[data['Date'].dt.year == 2013]
data2013.head()

In [ ]:
data['TypeEffet'].value_counts()
data.query('TypeEffet == "Bon de livraison" and CleEffet > "546"')
data.query('(PrixUnitaireTTC - PrixAchat) * Quantite != DetailEffet_Marge and TypeEffet == "Bon de livraison"' )




In [ ]:
data2013Livraison = data2013[data2013['TypeEffet'] == 'Bon de livraison']


# Extract month and year from the 'Date' column
data2013Livraison['Month'] = data2013Livraison['Date'].dt.month

# Group by 'FamilleArticle' and 'Month' and sum the 'Quantite' for each group
grouped = data2013Livraison.groupby(['FamilleArticle', 'Month'])['DetailEffet_Marge'].sum().reset_index()

# Splitting categories into groups of 10 for plotting
categories = grouped['FamilleArticle'].unique()

for i in range(len(categories) // 10 + 1):
    plt.figure(figsize=(10, 6))
    
    for j in range(i * 10, min((i + 1) * 10, len(categories))):
        plt.title(f'Categories {i * 10 + 1} to {min((i + 1) * 10, len(categories))}')
        
        category = categories[j]
        category_data = grouped[grouped['FamilleArticle'] == category]
        plt.plot(category_data['Month'], category_data['DetailEffet_Marge'], label=category)
        
        plt.xlabel('Months')
        plt.ylabel('DetailEffet_Marge')
    
    plt.legend()
    plt.tight_layout()
    
    # Save each set of 10 categories as an individual image file
    plt.savefig(f'plot_categories_{i * 10 + 1}_to_{min((i + 1) * 10, len(categories))}.png')
    plt.close()  # Close the current figure to prevent overlapping plots

print("Plots saved as individual image files.")

In [ ]:
# Define a custom function to sort the DataFrame within each group
def sort_by_detail_marge(group):
    return group.sort_values(by='DetailEffet_Marge', ascending=False)

# Apply the custom function to each group within each month
sorted_grouped = grouped.groupby('Month', group_keys=False).apply(sort_by_detail_marge)

# Path to save PDF files
pdf_path = 'tables2013'

# Iterate over each month
for month, month_group in sorted_grouped.groupby('Month'):
    # Calculate the percentage column and round to two decimal places
    month_group['Percentage'] = (month_group['DetailEffet_Marge'] / month_group['DetailEffet_Marge'].sum() * 100).round(2)

    # Round 'DetailEffet_Marge' to two decimal places
    month_group['DetailEffet_Marge'] = month_group['DetailEffet_Marge'].round(2)

    # Create a table using matplotlib
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')

    # Include 'Percentage' column in the table
    table_data = month_group[['FamilleArticle', 'DetailEffet_Marge', 'Percentage']].values
    col_labels = ['FamilleArticle', 'DetailEffet_Marge', 'Percentage']
    ax.table(cellText=table_data,
             colLabels=col_labels,
             cellLoc='center', loc='center')

    # Save the table as a PDF file
    pdf_filename = f'{pdf_path}/table_{month}.pdf'
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight')
    plt.close()

In [ ]:
data2013Livraison = data[data['TypeEffet'] == 'Bon de livraison']


# Extract month and year from the 'Date' column
data2013Livraison['Month'] = data2013Livraison['Date'].dt.month

# Group by 'FamilleArticle' and 'Month' and sum the 'Quantite' for each group
grouped = data2013Livraison.groupby(['FamilleArticle', 'Month'])['DetailEffet_Marge'].sum().reset_index()


# Define a custom function to sort the DataFrame within each group
def sort_by_detail_marge(group):
    return group.sort_values(by='DetailEffet_Marge', ascending=False)

# Apply the custom function to each group within each month
sorted_grouped = grouped.groupby('Month', group_keys=False).apply(sort_by_detail_marge)

# Path to save PDF files
pdf_path = 'tables'

# Iterate over each month
for month, month_group in sorted_grouped.groupby('Month'):
    # Calculate the percentage column and round to two decimal places
    month_group['Percentage'] = (month_group['DetailEffet_Marge'] / month_group['DetailEffet_Marge'].sum() * 100).round(2)

    # Round 'DetailEffet_Marge' to two decimal places
    month_group['DetailEffet_Marge'] = month_group['DetailEffet_Marge'].round(2)

    # Create a table using matplotlib
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')

    # Include 'Percentage' column in the table
    table_data = month_group[['FamilleArticle', 'DetailEffet_Marge', 'Percentage']].values
    col_labels = ['FamilleArticle', 'DetailEffet_Marge', 'Percentage']
    ax.table(cellText=table_data,
             colLabels=col_labels,
             cellLoc='center', loc='center')

    # Save the table as a PDF file
    pdf_filename = f'{pdf_path}/table_{month}.pdf'
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight')
    plt.close()


In [ ]:
data2013Livraison[data2013Livraison['FamilleArticle'] == 'Courgette_Hybride']
data2013Livraison.query('FamilleArticle == "Courgette_Hybride" and Effet_Marge < 0')

In [ ]:
correct_marge=data.query('(PrixUnitaireTTC - PrixAchat)  == DetailEffet_Marge/Quantite and TypeEffet == "Bon de livraison"' )
correct_marge['unitMargin'] = correct_marge['DetailEffet_Marge'] / correct_marge['Quantite']
averagebyfamille=correct_marge.groupby('FamilleArticle')['unitMargin'].mean().reset_index()
averagebyfamille

In [ ]:
fixedData = data[data['TypeEffet'] == 'Bon de livraison']
# Multiply 'averagebyfamille' by 'quantity' and override 'Detaileffetmarge'
fixedData['DetailEffet_Marge'] = fixedData['FamilleArticle'].map(averagebyfamille.set_index('FamilleArticle')['unitMargin']) * fixedData['Quantite']
fixedData


In [ ]:
# Extract month and year from the 'Date' column
fixedData['Month'] = fixedData['Date'].dt.month
fixedData['Year'] = fixedData['Date'].dt.year

# Group by 'FamilleArticle', 'Month', and 'Year' and sum the 'DetailEffet_Marge' for each group
grouped = fixedData.groupby(['FamilleArticle', 'Month', 'Year'])['DetailEffet_Marge'].sum().reset_index()

# Group by 'FamilleArticle' and 'Month' and calculate the average of the monthly sums
average_grouped = grouped.groupby(['FamilleArticle', 'Month'])['DetailEffet_Marge'].mean().reset_index()

# Define a custom function to sort the DataFrame within each group
def sort_by_detail_marge(group):
    return group.sort_values(by='DetailEffet_Marge', ascending=False)

# Apply the custom function to each group within each month
sorted_grouped = average_grouped.groupby('Month', group_keys=False).apply(sort_by_detail_marge)

# Path to save PDF files
pdf_path = 'summedtables'

# Iterate over each month
for month, month_group in sorted_grouped.groupby('Month'):
    # Calculate the percentage column and round to two decimal places
    month_group['Percentage'] = (month_group['DetailEffet_Marge'] / month_group['DetailEffet_Marge'].sum() * 100).round(2)

    # Round 'DetailEffet_Marge' to two decimal places
    month_group['DetailEffet_Marge'] = month_group['DetailEffet_Marge'].round(2)

    # Create a table using matplotlib
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')

    # Include 'Percentage' column in the table
    table_data = month_group[['FamilleArticle', 'DetailEffet_Marge', 'Percentage']].values
    col_labels = ['FamilleArticle', 'DetailEffet_Marge', 'Percentage']
    ax.table(cellText=table_data,
             colLabels=col_labels,
             cellLoc='center', loc='center')

    # Save the table as a PDF file
    pdf_filename = f'{pdf_path}/table_{month}.pdf'
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight')
    plt.close()

In [ ]:
# Group by 'FamilleArticle' and 'Year' and calculate the sum of 'DetailEffet_Marge' for each group
evolution_data = grouped.groupby(['FamilleArticle', 'Year'])['DetailEffet_Marge'].sum().reset_index()

# Create an evolution graph for each 'FamilleArticle'
for famille, famille_group in evolution_data.groupby('FamilleArticle'):
    plt.figure(figsize=(10, 6))
    plt.plot(famille_group['Year'], famille_group['DetailEffet_Marge'], marker='o', label=famille)
    plt.title(f'Evolution of DetailEffet_Marge for {famille} over Time')
    plt.xlabel('Year')
    plt.ylabel('DetailEffet_Marge Sum')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
%pip install bar_chart_race


In [ ]:
import bar_chart_race as bcr
plt.rcParams['animation.ffmpeg_path'] = '../ffmpeg-6.1'

# Assuming you have a DataFrame named 'fixedData'
# Make sure that the 'Date', 'FamilleArticle', and 'DetailEffet_Marge' columns are present in the DataFrame

# Extract month and year from the 'Date' column
fixedData['Month'] = fixedData['Date'].dt.month
fixedData['Year'] = fixedData['Date'].dt.year

# Group by 'FamilleArticle', 'Month', and 'Year' and sum the 'DetailEffet_Marge' for each group
grouped = fixedData.groupby(['FamilleArticle', 'Month', 'Year'])['DetailEffet_Marge'].sum().reset_index()

# Pivot the DataFrame to have 'Year' as index, 'FamilleArticle' as columns, and 'DetailEffet_Marge' as values
pivot_df = grouped.pivot_table(index='Year', columns='FamilleArticle', values='DetailEffet_Marge', fill_value=0)

# Create a bar chart race
bcr.bar_chart_race(
    df=pivot_df,
    filename='bar_chart_race_evolution.mp4',
    orientation='h',
    sort='desc',
    n_bars=10,
    fixed_order=False,
    steps_per_period=10,
    interpolate_period=True,
    period_length=500,
    figsize=(8, 5),
    title='Evolution of DetailEffet_Marge for FamilleArticles over Time'
)

### 1.3 Data Cleaning and Preprocessing:

<font color="#ffe9c4">At this stages we should improve the quality of our data, and make it easy to use and work with by doing the following:</font>

#### 1.3.1 Data Cleanning:
Clean the raw data by handling missing values, outliers, and inconsistencies.



#### 1.3.2 Data Preprocessing:
Preprocess the data to transform it into a suitable format (type) for analysis since most of our data are of type object. This step include also normalization, standardization, and encoding categorical variables and also transforming the attributes names and content to english since they are written in french.

### 1.4 Exploratory Data Analysis (EDA):

<font color="#ffe9c4">Explore the data to gain insights and an initial understanding of its characteristics. This may involve the use of statistical summaries, visualizations, and other exploratory techniques.</font>